In [1]:
from aicsimageio import AICSImage
import os

import pandas as pd
import numpy as np

from napari import Viewer

from tifffile import imread

In [2]:
file = "/home/tibuch/Data/gacho_treat-experiments-input/pPD12-C-1h/MS2-800_Ren-500_DAPI-50_5.nd"
nuc_cyto_seg_dir = "/home/tibuch/Data/gchao_treat-experiments-outputs/pPD12-C-1h/02_nuclei-cyto-segmentation/"
spot_dir = "/home/tibuch/Data/gchao_treat-experiments-outputs/pPD12-C-1h/03_spot-detection/"

In [3]:
name, ext = os.path.splitext(os.path.basename(file))

In [4]:
img = AICSImage(file)

In [5]:
raw_01 = img.data[0, 0]
raw_02 = img.data[0, 1]

/home/tibuch/Gitrepos/gchao_treat-experiments/infrastructure/apps/micromamba/envs/treat-exp/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
/home/tibuch/Gitrepos/gchao_treat-experiments/infrastructure/apps/micromamba/envs/treat-exp/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [6]:
nuc_seg = imread(os.path.join(nuc_cyto_seg_dir, name + '_NUC-SEG-3D.tif'))
cyto_seg = imread(os.path.join(nuc_cyto_seg_dir, name + '_CYTO-SEG-2D.tif'))

In [7]:
spots_01 = pd.read_csv(os.path.join(spot_dir, name + "_SPOTS_C01.csv"))
spots_02 = pd.read_csv(os.path.join(spot_dir, name + "_SPOTS_C02.csv"))

In [8]:
sx = img.ome_metadata.dict()['images'][0]['pixels']['physical_size_x']
sy = img.ome_metadata.dict()['images'][0]['pixels']['physical_size_y']
sz = img.ome_metadata.dict()['images'][0]['pixels']['physical_size_z']
spacing = tuple([sz, sy, sx])

/home/tibuch/Gitrepos/gchao_treat-experiments/infrastructure/apps/micromamba/envs/treat-exp/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [9]:
viewer = Viewer()

In [10]:
viewer.add_image(raw_01, scale=spacing, name="C01", contrast_limits=np.quantile(raw_01, [0.6, 0.999]), colormap="magenta")
viewer.add_image(raw_02, scale=spacing, name="C02", contrast_limits=np.quantile(raw_02, [0.6, 0.999]), colormap="green", blending='additive')
viewer.add_labels(cyto_seg, scale=spacing[1:], name="cyto")
viewer.layers["cyto"].contour = 1
viewer.add_labels(nuc_seg, scale=spacing, name="nuclei", opacity=0.5)

<Labels layer 'nuclei' at 0x7f5607890370>

In [11]:
viewer.add_points(spots_01.query("in_nuclei & symmetric")[["centroid_z", "centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='magenta',
                   face_color='#00000000', name="nuc-spots-01", symbol='o')

<Points layer 'nuc-spots-01' at 0x7f56052964f0>

In [12]:
viewer.add_points(spots_01.query("in_cyto & symmetric")[["centroid_z", "centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='magenta',
                   face_color='#00000000', name="cyto-spots-01", symbol='square')

<Points layer 'cyto-spots-01' at 0x7f56ccd9e340>

In [13]:
viewer.add_points(spots_02.query("in_nuclei & symmetric")[["centroid_z", "centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='green',
                   face_color='#00000000', name="nuc-spots-02", symbol='o')

<Points layer 'nuc-spots-02' at 0x7f561445b100>

In [14]:
viewer.add_points(spots_02.query("in_cyto & symmetric")[["centroid_z", "centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='green',
                   face_color='#00000000', name="cyto-spots-02", symbol='square')

<Points layer 'cyto-spots-02' at 0x7f5607fe6520>

In [302]:
#screenshot = viewer.screenshot(scale=5, path='./screenshot_slice.png')

In [309]:
viewer.add_image(np.max(raw_01, 0), scale=spacing[1:], name="C01", contrast_limits=np.quantile(np.max(raw_01, 0), [0.6, 0.999]), colormap="magenta")
viewer.add_image(np.max(raw_02, 0), scale=spacing[1:], name="C02", contrast_limits=np.quantile(np.max(raw_02, 0), [0.6, 0.999]), colormap="green", blending='additive')
viewer.add_labels(cyto_seg, scale=spacing[1:], name="cyto")
viewer.layers["cyto"].contour = 1
viewer.add_labels(np.max(nuc_seg, 0), scale=spacing[1:], name="nuclei", opacity=0.5)

<Labels layer 'nuclei' at 0x7fd2deef25b0>

In [310]:
viewer.add_points(spots_01.query("in_nuclei & symmetric")[["centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='magenta',
                   face_color='#00000000', name="nuc-spots-01", symbol='o')

<Points layer 'nuc-spots-01' at 0x7fd2ed01f5e0>

In [311]:
viewer.add_points(spots_01.query("in_cyto & symmetric")[["centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='magenta',
                   face_color='#00000000', name="cyto-spots-01", symbol='square')

<Points layer 'cyto-spots-01' at 0x7fd2e597c2b0>

In [312]:
viewer.add_points(spots_02.query("in_nuclei & symmetric")[["centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='green',
                   face_color='#00000000', name="nuc-spots-02", symbol='o')

<Points layer 'nuc-spots-02' at 0x7fd2dee320d0>

In [313]:
viewer.add_points(spots_02.query("in_cyto & symmetric")[["centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='green',
                   face_color='#00000000', name="cyto-spots-02", symbol='square')

<Points layer 'cyto-spots-02' at 0x7fd36b535c40>

In [314]:
#screenshot = viewer.screenshot(scale=5, path='./screenshot_projection.png')